# MENA Analysis: Disasters

### Load packages and setup

In [2]:
import pandas as pd
import geopandas as gpd
import pandas as pd
import numpy as np
import glob

In [3]:
# Define directories
dir_shp = '/Users/ashitakarl/Library/CloudStorage/Dropbox/WB/MENA_WorldBank/Boundaries/'
dir_in = '/Users/ashitakarl/Desktop/Project/WB/MENA/Disasters/'
dir_out = '/Users/ashitakarl/Library/CloudStorage/Dropbox/WB/MENA_WorldBank/Index/'

In [3]:
%matplotlib inline

### Load shapefile

In [12]:
# Load shapefile
MENA_shp = gpd.read_file(dir_shp + 'MENA_ADM2.shp')

MENA_shp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
flood_df = pd.read_excel(dir_in + 'FloodArchive.xlsx')[['long', 'lat', 'Began', 'Severity']]
flood_df['year'] = flood_df['Began'].astype(str).str[:4]

In [10]:
flood_df

,long,lat,Began,Severity,year
0,5.23026,35.81420,1985-01-01,1.0,1985
1,-45.34890,-18.71110,1985-01-15,2.0,1985
2,122.97400,10.02070,1985-01-20,1.0,1985
3,124.60600,1.01489,1985-02-04,1.0,1985
4,32.34910,-25.86930,1985-02-09,2.0,1985
...,...,...,...,...,...
5125,101.32300,17.32040,2021-09-23,1.5,2021
5126,-86.62910,32.49010,2021-10-06,1.5,2021
5127,-100.83200,19.95080,2021-10-01,1.5,2021
5128,87.96100,23.79840,2021-10-01,1.5,2021


### Flood

In [26]:
# Load flood data
flood_df = pd.read_excel(dir_in + 'FloodArchive.xlsx')[['long', 'lat', 'Began', 'Severity']]
flood_df['year'] = flood_df['Began'].astype(str).str[:4].astype(int)

# Convert df to shp
flood_shp = gpd.GeoDataFrame(flood_df, 
                             geometry = gpd.points_from_xy(flood_df.long, 
                                                           flood_df.lat, crs = "EPSG:4326"))

# Spatial join
flood_df = flood_shp.sjoin(MENA_shp, how = "left").groupby(['ID_ADM', 'year']).size().rename('n').reset_index()

# Filter by year
flood_df = flood_df.loc[(flood_df['year'] >= 1995) & (flood_df['year']<= 2020)]

# Define output FILE and export
VAR = 'FLOOD_'
YEAR_LIST = [1995, 2020]
FILE_OUT = 'ADM2_' + VAR + str(min(YEAR_LIST)) + '_' + str(max(YEAR_LIST)) + '.csv'

# Export
flood_df.to_csv(dir_out + FILE_OUT, index = False)

### Earthquake: mag >= 4.5

In [45]:
# Load flood data
quake_df = pd.read_csv(dir_in + 'Earthquake_USGS.csv')[['longitude', 'latitude', 'time']]
quake_df['year'] = quake_df['time'].astype(str).str[:4].astype(int)

# Convert df to shp
quake_shp = gpd.GeoDataFrame(quake_df, 
                             geometry = gpd.points_from_xy(quake_df.longitude, 
                                                           quake_df.latitude, crs = "EPSG:4326"))

# Spatial join
quake_df = quake_shp.sjoin(MENA_shp, how = "left").groupby(['ID_ADM', 'year']).size().rename('n').reset_index()

# Filter by year
quake_df = quake_df.loc[(quake_df['year'] >= 1995) & (quake_df['year']<= 2020)]

# Define output FILE and export
VAR = 'QUAKE_'
YEAR_LIST = [1995, 2020]
FILE_OUT = 'ADM2_' + VAR + str(min(YEAR_LIST)) + '_' + str(max(YEAR_LIST)) + '.csv'

# Export
quake_df.to_csv(dir_out + FILE_OUT, index = False)

### Landslides

In [44]:
# Load flood data
land_df = pd.read_csv(dir_in + 'nasa_global_landslide_catalog_point.csv')[['longitude', 'latitude', 'event_date']]
land_df = land_df[~land_df['event_date'].isnull()]
land_df['year'] = land_df['event_date'].astype(str).str[:4].astype(int)

# Convert df to shp
land_shp = gpd.GeoDataFrame(land_df, 
                            geometry = gpd.points_from_xy(land_df.longitude, 
                                                          land_df.latitude, crs = "EPSG:4326"))

# Spatial join
land_df = land_shp.sjoin(MENA_shp, how = "left").groupby(['ID_ADM', 'year']).size().rename('n').reset_index()

# Filter by year
land_df = land_df.loc[(land_df['year'] >= 1995) & (land_df['year']<= 2020)]

# Define output FILE and export
VAR = 'SLIDE_'
YEAR_LIST = [1995, 2020]
FILE_OUT = 'ADM2_' + VAR + str(min(YEAR_LIST)) + '_' + str(max(YEAR_LIST)) + '.csv'

# Export
land_df.to_csv(dir_out + FILE_OUT, index = False)

In [43]:
land_df

,ID_ADM,year,n
1,15549.0,2008,1
2,15549.0,2017,1
3,15560.0,2017,1
4,18311.0,2009,1
5,18315.0,2009,1
6,18797.0,2012,1
7,18799.0,2015,1
8,18806.0,2011,1
9,18807.0,2008,1
10,18807.0,2012,1
